Internet Resources:

[handson-ml/06_decision_trees.ipynb](https://github.com/ageron/handson-ml/blob/master/06_decision_trees.ipynb)  
[Sefik Ilkin Serengil - A Step by Step CART Decision Tree Example](https://sefiks.com/2018/08/27/a-step-by-step-cart-decision-tree-example/)  
[Google Developers - Tree Classifier from Scratch](https://www.youtube.com/watch?v=LDRbO9a6XPU&t=1s)  
[Victor Zhou - A Simple Explanation of Gini Impurity](https://victorzhou.com/blog/gini-impurity/)


Literature:  

Aurelien geron hands on machine learning page 173

In [1]:
%matplotlib inline
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import numpy as np

np.random.seed(42)

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# data set is the iris dataset from https://www.kaggle.com/arshid/iris-flower-dataset
# we are only keeping features "petal_length" and "petal_width"
df = pd.read_csv("data/iris.csv").drop(["sepal_width", "sepal_length"], 1)

# convert labels to numbers
label_mappings = {label_str:i for i,label_str in enumerate(df["species"].unique())}
df.replace({"species":label_mappings}, inplace=True)

# shuffle data
permutation = np.random.permutation(df.index)
X = np.array(df.drop(["species"], 1))[permutation]
y = np.array(df["species"])[permutation]

df

,petal_length,petal_width,species
0,1.4,0.2,0
1,1.4,0.2,0
2,1.3,0.2,0
3,1.5,0.2,0
4,1.4,0.2,0
...,...,...,...
145,5.2,2.3,2
146,5.0,1.9,2
147,5.2,2.0,2
148,5.4,2.3,2


## What is decision tree ##
Here's an example of a descision tree: the nodes hold questions, the last node of each branch (leaf nodes) holds a class, i.e prediction of the tree.
![](https://miro.medium.com/max/690/1*xzF10JmR3K0rnZ8jtIHI_g.png)

## How to build a decision tree ##
Our goal is to create a tree that best fits our data. We will be creating nodes recursivly.  

###### Creating Nodes ######
When creating a node our goal is to find a "question" (e.g petal_length > 3? salary > 50k? hair color == blond?) with which we can "best" split up our data into two parts. Mathematically speaking, "best" means it minimizes some cost function. We'll look at one cost function later. 

Here's an **outline** of what's to do in pseudocode:       

We consider every value in every feature for every sample in given training data as a potential question. For each potential question we split our data according to the question and calculate the cost. Our final question for the node is the one that produces the lowest cost. Then we create two new child nodes, each for one set of the split training data.
When we've reached the end of a branch we create a leaf node that will just reaturn the majority class of its training data when given a query

###### Cost function ######
The cost function used in this example is the gini index. It measures the likelihood of missclassifying a sample. It's defined as:  

$$ \text{gini} = \sum p_i \cdot (1-p_i) = 1 - \sum p_i^2 \in [0,1]$$

where $i$ is the ith class and $p_i = \frac{\text{# of samples in class i}}{\text{# of samples in all classes}}$ is the probability of of class $i$. 1 is the worst value, 0 the best.

In [10]:

def gini(distributions): 
    # distributions is a list where each element represents the amount of Samples belonging to each class
    # example [20, 30, 10] -> 20 x Sample belonging to class 0 
                        #  -> 30 x Sample belonging to class 1
                        #  -> 10 x Sample belonging to class 2
    if sum(distributions) == 0: # if there are no samples for the node
        return 0
    num_samples = sum(distributions)
    impurity = 1
    for i in distributions:
        impurity -= (i/num_samples)**2
    return impurity
 

# question function of node
def ask(descision_value, ask_value):
    if isinstance(ask_value, int) or isinstance(ask_value, float):    
        return ask_value >= descision_value
    else:
        return ask_value == descision_value


    
# end of branch
class TreeNodeEnd():
    def __init__(self, labels, distribution, depth):
        self.label = int(np.bincount(labels).argmax()) # most frequent label in labels
        self.distribution = distribution
        self.depth = depth
        self.gini_score = gini(self.distribution)
    
    def predict(self, value):
        return self.label # majority class of training data
    
    def print_tree(self):
        print("\t"*self.depth, "Sample is of class {}".format(self.label))
    
    
    
    
# decision node
class TreeNode():
    def __init__(self, data, labels, distrb, gini, descision_value, feature, depth):
        self.data = data
        self.labels = labels
        self.distribution = distrb
        self.gini_score = gini
        self.feature = feature
        self.descision_value = descision_value
        self.depth = depth
        self.child = {True:None, False:None}
    
    def predict(self, value):            
        return self.child[ask(self.descision_value, value[self.feature])].predict(value)
        
    def print_tree(self):
        cond = "is greater or equal to" if isinstance(self.descision_value, int) or isinstance(self.descision_value, float) else "is equal to" 
        print("\t"*self.depth, "if feature {} {} {}:".format(self.feature, cond, self.descision_value))
        self.child[True].print_tree()
        print("\t"*self.depth, "else:")
        self.child[False].print_tree()
   

        
    
        
class DescisionTreeClassifier:
    def __init__(self, max_depth = None):
        self.max_depth = max_depth
    
    def find_lowest_gini(self, data, labels):
        # find the featrue value (=question) that produces the lowest gini score and partition the data according to it
        
        # only get unique feature values - no need to check the same feature value twice
        unique_column_values = {i:list(np.unique(column)) for i,column in enumerate(data.T) } 
        gini_score = 1
        
        # for each feature
        for column in unique_column_values:
            # for each value in this feature
            for unique_value in unique_column_values[column]: 
                # distrb is a list where each element represents the amount of Samples belonging to each class
                # example [20, 30, 10] -> 20 x Sample belonging to class 0 
                        #  -> 30 x Sample belonging to class 1
                        #  -> 10 x Sample belonging to class 2
                distrb = {True:[0] * self.class_count, False:[0] * self.class_count}
                
                # split data into two groups: True and False
                split_data = {True:[], False:[]}
                split_labels = {True:[], False:[]}
                for i in range(len(data)):
                    # evaluate question with current feature value
                    branch = ask(unique_value, data.T[column][i])
                    split_data[branch].append(data[i])
                    split_labels[branch].append(labels[i])
                    distrb[branch][labels[i]] += 1
                
                # gini score is the weighted sum of both branches
                new_gini = (sum(distrb[True]) / len(data) * gini(distrb[True])) + (sum(distrb[False]) \
                                                                                   / len(data) * gini(distrb[False]))
                
                #convert from list to np.array
                split_data = {True:np.array(split_data[True]), False:np.array(split_data[False])}
                split_labels = {True:np.array(split_labels[True]), False:np.array(split_labels[False])}
                
                # check if question has lower gini score
                if new_gini < gini_score:
                    gini_score = new_gini
                    # split data and labels, distribution, question value, feature
                    lowest_gini_data = split_data, split_labels, distrb, gini_score, unique_value, column
        return lowest_gini_data 
            
    
    def create_next_node(self, data, labels, distribution, depth=0):
        # recursive function, builds the tree
        label = int(np.bincount(labels).argmax()) # most often occouring label
        
        # feature value that produces the lowest gini score and partition data accordinglý
        split_data, split_labels, distrb, gini_score, descision_value, feature = self.find_lowest_gini(data, labels)
        
        # data has to be splitable 
        if list(distrb[True]) != distribution and list(distrb[False]) != distribution:
            node = TreeNode(data, labels, distribution, gini_score, descision_value, feature, depth)
            
            # need to create nodes for both outcomes 
            for branch in [True, False]:
                # check if depth has been reched
                depth_reached = False if self.max_depth is None else True if depth+1 == self.max_depth else False
                # node is pure if all training data belongs to one class only
                node_pure = distrb[branch].count(0) >= self.class_count-1
                # create child nodes
                if depth_reached or node_pure:
                    node.child[branch] = TreeNodeEnd(split_labels[branch], distrb[False], depth+1)
                else:
                    # recursion
                    node.child[branch] = self.create_next_node(split_data[branch], split_labels[branch], distrb[branch], depth+1)
            
        else: # otherwise we are at the end of the branch
            node = TreeNodeEnd(labels, distribution, depth)
        return node
        
    
    def fit(self, data, labels):
        # start recursive process of creating nodes
        self.class_count = len(np.unique(labels))
        distribution = [0] * len(np.unique(labels))
        for i in range(len(labels)):
            distribution[labels[i]] += 1
        self.root = self.createNextNode(data, labels, distribution)
        
    def predict(self, sample):
        # recursive function, predicts class of sample
        return self.root.predict(sample)
    
    def print_tree(self):
        # recursive function, prints decision questions of the tree
        self.root.print_tree()
    
    
    
clf = DescisionTreeClassifier(max_depth=None)
clf.fit(X, y)
clf.print_tree()
clf.predict([1,2])

 if feature 0 is greater or equal to 3.0:
	 if feature 1 is greater or equal to 1.8:
		 if feature 0 is greater or equal to 4.9:
			 Sample is of class 2
		 else:
			 Sample is of class 2
	 else:
		 if feature 0 is greater or equal to 5.0:
			 if feature 1 is greater or equal to 1.6:
				 if feature 0 is greater or equal to 5.8:
					 Sample is of class 2
				 else:
					 Sample is of class 1
			 else:
				 Sample is of class 2
		 else:
			 if feature 1 is greater or equal to 1.7:
				 Sample is of class 2
			 else:
				 Sample is of class 1
 else:
	 Sample is of class 0


0